In [1]:
import pandas as pd
import tabula
import os
from PyPDF2 import PdfReader
import subprocess
import numpy as np

In [2]:
# get all files in a folder
# folder is where data of a give year are stored
folder = './ceap_2022'
file_names = os.listdir(folder)
len(file_names)

506

### Functions

In [890]:
def _shear_names( str_name ):
    
    #strip word vereador: and split first name and last names
    names = str_name.split(" ", 1)[1]
    names = names.split(" ", 1)
    
    return names

def get_specific_line(string, line_number):
    """
    This function returns the specified line from a string separated by new lines.
    If the line number is out of range, it returns None.
    """
    lines = string.split('\n')
    
    if 0 <= line_number < len(lines):
        return lines[line_number]
    else:
        return None
    
def get_name(index, line):
    """
    This function extracts the name of the deputy from the PDF file.
    """
    with open(folder+'/'+file_names[index], 'rb') as file:
        reader = PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
        return get_specific_line(text, line).split(':')[1]

def open_pdf(index):
    """
    This function opens the PDF file in the default PDF viewer.
    """
    subprocess.Popen(['evince', folder+'/'+file_names[index]])

def has_duplicates(input_list):
    """
    This function checks if a list has repeated elements.
    :param input_list: List of elements to check
    :return: True if there are duplicates, False otherwise
    """
    return len(input_list) != len(set(input_list))

def check_last_row_total(df):
    """
    This function checks if the last row of the dataframe is a total row.
    """
    nan_list = [np.nan, np.nan, np.nan, np.nan, 'TOTAL:']
    df_list = df.iloc[-1].to_list()[0:5]

    if len(nan_list) != len(df_list):
        return False
    for a, b in zip(nan_list, df_list):
        if a is np.nan and b is np.nan:
            continue
        if a != b:
            return False
    return True

def _check_type( df ):
    """
    Returns the number of columns
    7: column 2 with Data Documento and Nro.Documento together
    8: already correct, just eliminates cols 6-7
    10: those with two columns between Data Documento and Nro.Documento
    """

    return len( list(df.columns) )

def __columns_names_equal( df ):
    """
    Returns True if the columns names are equal to the expected ones
    """
    columns = ['CNPJ', 'Data Documento', 'Nro.Documento', 'Fornecedor', 'Item de Despesa']
    df_list = list(df.columns)[0:5]
    
    if len(columns) != len(df_list):
        return False
    for a, b in zip(columns, df_list):
        if a != b:
            return False
    
    return True

def _check_rows_number( df ):
    """
    Returns the number of rows to be deleted
    """
    try:
        rows = df[df.iloc[:, 0] == 'CNPJ'].index[0]
    except( IndexError):
        rows = 0

    return rows

### Opens a pdf file from the list

In [918]:
index = 399
#index 12 de 2021 tem pdf em branco, ceap 2022 done up to 156
try:
    dfs = tabula.read_pdf( folder+'/'+file_names[index], pages='all', pandas_options={'dtype':'str'}) # entrega um Dataframe
    #print(dfs[0])
except Exception:
    print("File doesn't exist")
    dfs = None
#open_pdf(index)
dfs[0]

Got stderr: mar 31, 2025 1:24:34 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
mar 31, 2025 1:24:34 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



,SIGAE - CEAP,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,08/06/2022
0,NaN,NaN,NaN,NaN,Demonstrativo Cota CEAP,NaN,NaN,Página: 1
1,Vereador: DAVID VALENTE REIS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Período:,á,NaN,NaN,NaN,Cota Mensal,"33.085,85",NaN
3,Situação: REGULAR,NaN,NaN,NaN,NaN,Sobra Mês Anterior:,"30.621,27",NaN
4,Competência: 05/2022,NaN,NaN,NaN,NaN,Outros Reeembolsos:,"0,00",NaN
5,NaN,NaN,NaN,NaN,NaN,Saldo Atual:,"63.707,12",NaN
6,Processo: 2022.10000.10718.0.001013,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,CNPJ,Data Documento,Nro.Documento,Fornecedor,Item de Despesa,Valor (R$),"63.707,12",(%)
8,23.505.231/0001-52,03/05/2022,94516,MANAUS COMERCIO DE,COMBUSTIVEL,"250,00","63.457,12","0,39"
9,23.505.231/0001-52,05/05/2022,94682,MANAUS COMERCIO DE,COMBUSTIVEL,"460,66","62.996,46","0,72"


### Format DataFrame

In [919]:
for i, df in enumerate(dfs):
    # verifica o tipo de dataframe
    df_type = _check_type( df )
    df = df.drop(df.columns[-2:], axis=1)

    # resolve o caso em que a coluna 2 tem Data Documento e Nro.Documento juntos
    if df_type == 7:
        df.columns = ['CNPJ', 'Data Documento', 'Fornecedor', 'Item de Despesa', 'Valor (R$)']
        # insere a coluna Nro.Documento
        df.insert(2, 'Nro.Documento', '')
        df_split = df['Data Documento'].str.split(' ', n=1, expand=True)
        df['Data Documento'] = df_split[0]
        df['Nro.Documento'] = df_split[1]

    elif df_type == 10:
        df.drop( df.columns[2:4], axis=1, inplace=True )
        df.columns = ['CNPJ', 'Data Documento', 'Nro.Documento', 'Fornecedor', 'Item de Despesa', 'Valor (R$)']
    
    else:
        df.columns = ['CNPJ', 'Data Documento', 'Nro.Documento', 'Fornecedor', 'Item de Despesa', 'Valor (R$)']

    remove_lines = _check_rows_number( df )
    if remove_lines > 0:
        # remove linhas que não são do dataframe
        df.drop( range(0, remove_lines+1), inplace=True )

    if check_last_row_total( df ):
        # se sim, apaga a última linha
        df = df.head(-1)

    name = get_name(index, 3)
    df.insert(0, 'Nome', name )
    
    dfs[i] = df
dfs[0]

,Nome,CNPJ,Data Documento,Nro.Documento,Fornecedor,Item de Despesa,Valor (R$)
8,DAVID VALENTE REIS,23.505.231/0001-52,03/05/2022,94516,MANAUS COMERCIO DE,COMBUSTIVEL,"250,00"
9,DAVID VALENTE REIS,23.505.231/0001-52,05/05/2022,94682,MANAUS COMERCIO DE,COMBUSTIVEL,"460,66"
10,DAVID VALENTE REIS,23.505.231/0001-52,05/05/2022,94678,MANAUS COMERCIO DE,COMBUSTIVEL,"318,72"
11,DAVID VALENTE REIS,23.505.231/0001-52,06/05/2022,94730,MANAUS COMERCIO DE,COMBUSTIVEL,"431,50"
12,DAVID VALENTE REIS,23.505.231/0001-52,08/05/2022,94869,MANAUS COMERCIO DE,COMBUSTIVEL,"323,38"
13,DAVID VALENTE REIS,23.505.231/0001-52,09/05/2022,94921,MANAUS COMERCIO DE,COMBUSTIVEL,"350,00"
14,DAVID VALENTE REIS,23.505.231/0001-52,10/05/2022,95000,MANAUS COMERCIO DE,COMBUSTIVEL,"300,00"
15,DAVID VALENTE REIS,23.505.231/0001-52,11/05/2022,95047,MANAUS COMERCIO DE,COMBUSTIVEL,"250,04"
16,DAVID VALENTE REIS,02.421.421/0031-37,11/05/2022,004.276.211-BB,TIM CELULAR S. A.,TELEFONIA MOVEL,"85,99"
17,DAVID VALENTE REIS,23.505.231/0001-52,12/05/2022,95154,MANAUS COMERCIO DE,COMBUSTIVEL,"302,39"


### Format Data Frame (when Data Documento and Nro.Documento comes in the same column)

In [719]:
lines_above = 9
pd.options.mode.copy_on_write = True
for i, df in enumerate(dfs):
    # armazenar o nome do vereador
    names = get_name(index, 3)
    column_names = ['CNPJ', 'Data Documento', 'Fornecedor', 'Item de Despesa', 'Valor (R$)']
    # procurar a linha que se encontra o titulo CNPJ

        
    df.drop( range(0, lines_above), inplace=True )
    df.drop( df.columns[5:], axis=1, inplace=True )
    df.columns = column_names
    df.insert(2, 'Nro.Documento', '')
    
    df_split = df['Data Documento'].str.split(' ', n=1, expand=True)
    df['Data Documento'] = df_split[0]
    df['Nro.Documento'] = df_split[1]
    #delete last line if it is Total line
    if check_last_row_total(df):
        df   =   df.head( -1 )
    df.insert(0, 'Nome', names )

    dfs[i] = df
dfs[0]

,Nome,CNPJ,Data Documento,Nro.Documento,Fornecedor,Item de Despesa,Valor (R$)
9,LUIS AUGUSTO MITOSO JUNIOR,02.558.157/0001-62,20/10/2022,102001,TELEFONICA BRASIL S.A.,TELEFONIA MOVEL,"1.279,99"
10,LUIS AUGUSTO MITOSO JUNIOR,19.713.467/0001-07,20/10/2022,A18,SUPORTE LOCADORA DE,LOC.VEICULO TERRESTRE FLUVIAL,"11.500,00"
11,LUIS AUGUSTO MITOSO JUNIOR,84.472.851/0003-99,20/10/2022,2844,PETROVAN DERIVADOS DE,COMBUSTIVEL,"14.900,00"
12,LUIS AUGUSTO MITOSO JUNIOR,17.458.949/0001-32,21/10/2022,294,GRAFICA ZILO,DIVUL. ATIVIDADE PARLAMENTAR,"17.000,00"


### Format Data Frame (Simple Version)

In [759]:
# all 2021 CEAP files have the same structure
for i, df in enumerate(dfs):
    # armazebnar o nome do vereador
    name = get_name(index, 3)

    # delete last line if it is Total line
    if check_last_row_total(df):
        df   =   df.head( -1 )
    df.drop( df.columns[[6,7]], axis=1, inplace=True)
    df.insert(0, 'Nome', name)

    dfs[i] = df

dfs[0]

,Nome,CNPJ,Data Documento,Nro.Documento,Fornecedor,Item de Despesa,Valor (R$)
0,ALLAN CAMPELO DA SILVA,01.691.433/0001-01,02/03/2022,3848,PETROMAY DERIVADOS DE,COMBUSTIVEL,"488,91"
1,ALLAN CAMPELO DA SILVA,04.643.904/0011-68,14/03/2022,201033,AUTO POSTO OZIVAL XI,COMBUSTIVEL,"100,00"
2,ALLAN CAMPELO DA SILVA,31.229.178/0002-13,14/03/2022,41600,DIAMANTE COMERCIO DE,COMBUSTIVEL,"190,00"
3,ALLAN CAMPELO DA SILVA,19.943.959/0001-99,21/03/2022,1229,R K TRANSPORTE LOGISTICA,LOC.VEICULO TERRESTRE FLUVIAL,"9.000,00"
4,ALLAN CAMPELO DA SILVA,17.199.693/0001-96,22/03/2022,2947,R A DE LIMA EIRELI ME,DIVUL. ATIVIDADE PARLAMENTAR,"5.000,00"
5,ALLAN CAMPELO DA SILVA,26.928.710/0001-70,22/03/2022,104223,POSTO TORQUATO COMERCIO,COMBUSTIVEL,"1.484,89"
6,ALLAN CAMPELO DA SILVA,02.558.157/0001-62,23/03/2022,20220323812654,TELEFONICA BRASIL S.A.,TELEFONIA MOVEL,"176,90"
7,ALLAN CAMPELO DA SILVA,02.558.157/0001-62,23/03/2022,20220323814438,TELEFONICA BRASIL S.A.,TELEFONIA MOVEL,"247,79"
8,ALLAN CAMPELO DA SILVA,10.988.014/0001-19,23/03/2022,550930,AMAZON COMBUSTÍVEIS PARA,COMBUSTIVEL,"653,25"
9,ALLAN CAMPELO DA SILVA,19.470.625/0001-45,24/03/2022,000164108,AMAZON EMPREENDIMENTOS,COMBUSTIVEL,"473,00"


### Format DataFrame (Those whith 1 column between Data Documento and Nro.Documento)

In [231]:
pd.options.mode.copy_on_write = True
lines_above =   5
for i, df in enumerate(dfs):
    # stores congressman name
    names = get_name(index, 3)
    column_names = ['CNPJ', 'Data Documento', 'Nro.Documento', 'Fornecedor', 'Item de Despesa', 'Valor (R$)']

    # Delete columns between Data Documento and Nro.Documento
    df.drop( df.columns[2], axis=1, inplace=True )
    # Delete everything above first data entry
    df.drop( range(0,lines_above), inplace=True )
    # Delete columns after Valor (R$)
    df.drop( df.columns[6:], axis=1, inplace=True )
    # Rename columns
    df.columns = column_names

    # delete last line if it is Total line
    if check_last_row_total(df):
        df   =   df.head( -1 )

    # Insert name
    df.insert(0, 'Nome', names )

    dfs[i] = df
dfs[0]

,Nome,CNPJ,Data Documento,Nro.Documento,Fornecedor,Item de Despesa,Valor (R$)
5,JOAO KENNEDY DE LIMA MARQUES,05.220.251/0005-29,19/10/2022,1092,CIDADE COMERCIO DE,COMBUSTIVEL,"9.003,97"
6,JOAO KENNEDY DE LIMA MARQUES,43.018.000/0001-21,20/10/2022,002009,DUBAI AUTO PADRAO EIRELI,LOC.VEICULO TERRESTRE FLUVIAL,"6.000,00"
7,JOAO KENNEDY DE LIMA MARQUES,008.949.082-78,20/10/2022,20223325622,LUCIANA DE SOUZA DOS SANTOS,ASSESS.CONSUL TECNICA,"12.000,00"


### Format DataFrame (With 2 columns betwen Data Documento and Nro.Documento)

In [735]:
pd.options.mode.copy_on_write = True
for i, df in enumerate(dfs):
    name = get_name(index, 3)
    column_names = ['CNPJ', 'Data Documento', 'Nro.Documento', 'Fornecedor', 'Item de Despesa', 'Valor (R$)']
    df.drop( range(0,5), inplace=True )
    df.drop( df.columns[2:4], axis=1, inplace=True )
    df.drop( df.columns[6:], axis=1, inplace=True )
    df.columns = column_names

    # delete last line if it is Total line 
    if check_last_row_total(df):
        df   =   df.head( -1 )

    df.insert(0, 'Nome', name)

    dfs[i] = df
dfs[0]

,Nome,CNPJ,Data Documento,Nro.Documento,Fornecedor,Item de Despesa,Valor (R$)
5,IVO SANTOS DA SILVA NETO,43.498.752/0001-37,20/07/2022,60,A A RUSSO SERVIÇOS,ASSESS.CONSUL TECNICA,"8.400,00"
6,IVO SANTOS DA SILVA NETO,29.080.961/0001-35,30/07/2022,283,MAURICIO S LIMA ME,DIVUL. ATIVIDADE PARLAMENTAR,"9.800,00"
7,IVO SANTOS DA SILVA NETO,39.494.003/0001-64,30/07/2022,50,BARRETO E LIBORIO SOCIEDADE,ASSESS.CONSUL TECNICA,"8.000,00"


### Format DataFrame (those with all columns correct but with a number of lines to be deleted)

In [755]:
lines_above = 8
for i, df in enumerate(dfs):
    # stores congressman name
    names = get_name(index, 3)
    column_names = ['CNPJ', 'Data Documento', 'Nro.Documento', 'Fornecedor', 'Item de Despesa', 'Valor (R$)']
   
    df.drop( range(0, lines_above), inplace=True )
    df.drop( df.columns[6:], axis=1, inplace=True )
    df.columns = column_names

    if check_last_row_total(df):
        df   =   df.head( -1 )

    df.insert(0, 'Nome', names )

    dfs[i] = df
dfs[0]

,Nome,CNPJ,Data Documento,Nro.Documento,Fornecedor,Item de Despesa,Valor (R$)
8,DAVID VALENTE REIS,23.505.231/0001-52,03/05/2022,94516,MANAUS COMERCIO DE,COMBUSTIVEL,"250,00"
9,DAVID VALENTE REIS,23.505.231/0001-52,05/05/2022,94682,MANAUS COMERCIO DE,COMBUSTIVEL,"460,66"
10,DAVID VALENTE REIS,23.505.231/0001-52,05/05/2022,94678,MANAUS COMERCIO DE,COMBUSTIVEL,"318,72"
11,DAVID VALENTE REIS,23.505.231/0001-52,06/05/2022,94730,MANAUS COMERCIO DE,COMBUSTIVEL,"431,50"
12,DAVID VALENTE REIS,23.505.231/0001-52,08/05/2022,94869,MANAUS COMERCIO DE,COMBUSTIVEL,"323,38"
13,DAVID VALENTE REIS,23.505.231/0001-52,09/05/2022,94921,MANAUS COMERCIO DE,COMBUSTIVEL,"350,00"
14,DAVID VALENTE REIS,23.505.231/0001-52,10/05/2022,95000,MANAUS COMERCIO DE,COMBUSTIVEL,"300,00"
15,DAVID VALENTE REIS,23.505.231/0001-52,11/05/2022,95047,MANAUS COMERCIO DE,COMBUSTIVEL,"250,04"
16,DAVID VALENTE REIS,02.421.421/0031-37,11/05/2022,004.276.211-BB,TIM CELULAR S. A.,TELEFONIA MOVEL,"85,99"
17,DAVID VALENTE REIS,23.505.231/0001-52,12/05/2022,95154,MANAUS COMERCIO DE,COMBUSTIVEL,"302,39"


### Append to dfs

In [761]:
for df in dfs:
    MasterDF =  pd.concat( [MasterDF, df], ignore_index=True)

### Create Master DF based in the fisrt DataFrame

In [48]:
MasterDF = pd.DataFrame()

for df in dfs:
    MasterDF = pd.concat([MasterDF, df], ignore_index=True)


In [ ]:
backup = MasterDF.copy() # last 93

In [588]:
MasterDF = backup.copy()

### Visualization

In [777]:
#Just visizualize Master DaytaFrame
MasterDF.tail(35)

,Nome,CNPJ,Data Documento,Nro.Documento,Fornecedor,Item de Despesa,Valor (R$)
1581,ALLAN CAMPELO DA SILVA,11.220.341/0001-99,23/11/2022,1711,BATARA COMERCIO DE,COMBUSTIVEL,"14.515,00"
1582,ALLAN CAMPELO DA SILVA,17.199.693/0001-96,23/11/2022,3506,R A DE LIMA EIRELI ME,DIVUL. ATIVIDADE PARLAMENTAR,"16.000,00"
1583,ALLAN CAMPELO DA SILVA,02.558.157/0001-62,23/11/2022,261156730,TELEFONICA BRASIL S.A.,TELEFONIA MOVEL,"219,16"
1584,ALLAN CAMPELO DA SILVA,02.558.157/0001-62,23/11/2022,15849212130,TELEFONICA BRASIL S.A.,TELEFONIA MOVEL,"156,84"
1585,ALLAN CAMPELO DA SILVA,518.758.102-34,26/11/2022,20223335940,JACK GOMES DE SOUZA,ASSESS.CONSUL TECNICA,"9.000,00"
1586,ROBSON DA SILVA TEIXEIRA,84.534.346/0001-60,19/10/2022,460,WAUDECY BATISTA DO,DIVUL. ATIVIDADE PARLAMENTAR,"16.200,00"
1587,ROBSON DA SILVA TEIXEIRA,32.858.548/0001-63,20/10/2022,31,ROQUE LANE WILKENS,ASSESS.CONSUL TECNICA,"7.500,00"
1588,ROBSON DA SILVA TEIXEIRA,28.100.688/0001-09,20/10/2022,004-D/2022,ALPHA PRODUCOES DE EVENTOS,LOC.VEICULO TERRESTRE FLUVIAL,"14.400,00"
1589,JOAO KENNEDY DE LIMA MARQUES,792.908.412-49,20/06/2022,20223290485,SARAH GEORGIA DE FIGUEIREDO,ASSESS.CONSUL TECNICA,"3.600,00"
1590,JOAO KENNEDY DE LIMA MARQUES,05.220.251/0005-29,20/06/2022,917,CIDADE COMERCIO DE,COMBUSTIVEL,"6.700,51"


### Manual corrections

In [215]:
# delete last row
MasterDF.drop( MasterDF.index[-1], inplace=True)

In [596]:
MasterDF.drop(MasterDF.columns[[7,8]], axis=1, inplace=True)

In [333]:
MasterDF.loc[len(MasterDF)] = ['DAVID VALENTE REIS',
                               '30.546.101/0001-23',
                            '31/08/2022,',
                            '02-22',
                            'VITECH SERVICOS DE MONT DE',
                            'LOC.VEICULO TERRESTRE FLUVIAL',
                            '8.000,00']

In [548]:
#APENAS OS QUE NAO DECLARARAM GASTOS
MasterDF.loc[len(MasterDF)] = ['RODRIGO GUEDES OLIVEIRA DE ARAUJO',
                               '00.000.000/0000-00',
                               '01/09/2022',
                               '0',
                               'NAO USA EM PERIDO ELEITORAL',
                               'COMBUSTIVEL',
                               '0,00']

### Save

In [647]:
# Save DataFrame to a Pickle file (last save was index 373 - 2022)
MasterDF.to_pickle(folder+'.pkl')

### Load

In [776]:
# Load DataFrame from a Pickle file
MasterDF = pd.read_pickle(folder+'.pkl')

### Convert to CSV

In [18]:
MasterDF.to_csv(folder+'.csv', index=False, sep=';')

In [7]:
result = MasterDF.query('Nome.str.contains("JOÃO CARLOS DOS SANTOS MELO")', engine='python')
print( result )

                            Nome                CNPJ Data Documento  \
705  JOÃO CARLOS DOS SANTOS MELO  23.586.479/0001-95     30/01/2021   

    Nro.Documento                  Fornecedor                Item de Despesa  \
705           540  DS CAR ALUGUEL DE CARROS E  LOC.VEICULO TERRESTRE FLUVIAL   

    Valor (R$)  
705   3.000,00  


In [8]:
result = MasterDF.loc[MasterDF['Nome'] == 'JOÃO CARLOS DOS SANTOS MELLO']
print( result )

                              Nome                CNPJ Data Documento  \
277   JOÃO CARLOS DOS SANTOS MELLO  02.558.157/0001-62     10/08/2021   
278   JOÃO CARLOS DOS SANTOS MELLO  32.479.604/0001-59     18/08/2021   
279   JOÃO CARLOS DOS SANTOS MELLO      603.161.012-49     19/08/2021   
280   JOÃO CARLOS DOS SANTOS MELLO  37.212.905/0001-35     20/08/2021   
281   JOÃO CARLOS DOS SANTOS MELLO  10.353.166/0001-45     24/08/2021   
282   JOÃO CARLOS DOS SANTOS MELLO  10.583.553/0002-59     27/08/2021   
349   JOÃO CARLOS DOS SANTOS MELLO  02.558.157/0001-62     10/03/2021   
350   JOÃO CARLOS DOS SANTOS MELLO      603.161.012-49     25/03/2021   
351   JOÃO CARLOS DOS SANTOS MELLO  32.479.604/0001-59     27/03/2021   
352   JOÃO CARLOS DOS SANTOS MELLO  24.681.257/0001-14     27/03/2021   
353   JOÃO CARLOS DOS SANTOS MELLO  10.353.166/0001-45     27/03/2021   
354   JOÃO CARLOS DOS SANTOS MELLO  23.586.479/0001-95     29/03/2021   
705   JOÃO CARLOS DOS SANTOS MELLO  23.586.479/0001

In [5]:
print( MasterDF.columns )

Index(['Nome', 'CNPJ', 'Data Documento', 'Nro.Documento', 'Fornecedor',
       'Item de Despesa', 'Valor (R$)'],
      dtype='object')


In [10]:
array = MasterDF.loc[705].values
print( array )

['JOÃO CARLOS DOS SANTOS MELLO' '23.586.479/0001-95' '30/01/2021' '540'
 'DS CAR ALUGUEL DE CARROS E' 'LOC.VEICULO TERRESTRE FLUVIAL' '3.000,00']
